In [2]:
import pandas as pd

contest_csv = pd.read_csv("./contests.csv")
contest_csv

,contest_id,subject_type,dir_path,contest_name,contest_description,metadata,contest_start_date
0,1,NaN,./data/contests/1/,사진 대결\n갤럭시S25U vs 아이폰16PM,아이폰 16 프로맥스와 갤럭시 S24 울트라로 촬영한 결과물입니다. 뭐가 더 좋아보...,NaN,24.9.24.
1,2,NaN,./data/contests/2/,영상 대결\n갤럭시S25U vs 아이폰16PM,아이폰 16 프로맥스와 갤럭시 S24 울트라로 촬영한 결과물입니다. 뭐가 더 좋아보...,NaN,24.9.24.


In [5]:
# Update contest name for contest_id 1
contest_csv.loc[contest_csv['contest_id'] == 1, 'contest_name'] = "사진 대결\n갤럭시S25U vs 아이폰16PM"
contest_csv.loc[contest_csv['contest_id'] == 2, 'contest_name'] = "영상 대결\n갤럭시S25U vs 아이폰16PM"
contest_csv.to_csv("./contests.csv", index=False)


In [ ]:
python ./utils/metadata_handler.py ./data/contests/

In [5]:
def add_lens_tags(metadata_csv):
    aperture_to_tag_dict = {
        "iPhone 16 Pro Max": {
            1.78: "광각",
            1.9: "셀카", 
            2.2: "초광각", 
            2.8: "망원"
        },
        "Galaxy S25 Ultra": {
            1.7: "광각",
            1.9: "초광각", 
            2.2: "셀카",
            3.4: "망원",
            2.4: "망원"
        }
    }

    # Create a new 'tags' column initialized with empty lists
    metadata_csv['tags'] = [[] for _ in range(len(metadata_csv))]

    # Iterate through rows
    for idx, row in metadata_csv.iterrows():
        model = row['Model']
        aperture = round(row['Composite:Aperture'], 2) if pd.notna(row['Composite:Aperture']) else None
        
        # Only process if we have both model and aperture
        if pd.notna(model) and aperture is not None:
            # Get the aperture mapping for this model
            if model in aperture_to_tag_dict:
                model_apertures = aperture_to_tag_dict[model]
                # Find matching aperture and add corresponding tag
                for ap, tag in model_apertures.items():
                    if abs(ap - aperture) < 0.001:  # Compare with small tolerance
                        metadata_csv.at[idx, 'tags'].append(tag)
    
    return metadata_csv



In [ ]:
DIR = "./contests/1/"

metadata_csv = pd.read_csv(DIR+"metadata.csv")
# sort by CreateDate
metadata_csv = metadata_csv.sort_values("CreateDate")

metadata_csv = add_lens_tags(metadata_csv)

In [9]:
# Add "야간" tag for photos taken after 20:00
for idx, row in metadata_csv.iterrows():
    if pd.notna(row['CreateDate']):
        # Extract hour from CreateDate string (format: YYYY:MM:DD HH:MM:SS)
        hour = int(row['CreateDate'].split(' ')[1].split(':')[0])
        if hour >= 20:  # If time is 20:00 or later
            metadata_csv.at[idx, 'tags'].append('야간')

# Display the updated DataFrame
metadata_csv.to_csv(DIR+"metadata.csv")


In [ ]:
# Get unique match numbers, excluding NaN
match_numbers = metadata_csv['Match'].dropna().unique()

# For each match number
for match in match_numbers:
    # Get the rows for this match
    match_rows = metadata_csv[metadata_csv['Match'] == match]
    
    if len(match_rows) == 2:  # Ensure we have a pair
        print(f"\nMatch {int(match)}:")
        print(f"File: {match_rows['FileName'].iloc[0]}, Aperture: {match_rows['Composite:Aperture'].iloc[0]}")
        print(f"File: {match_rows['FileName'].iloc[1]}, Aperture: {match_rows['Composite:Aperture'].iloc[1]}")


        # Get unique aperture pairs for this match
        apertures = sorted(match_rows['Composite:Aperture'].unique())
        if len(apertures) > 1:  # Only print if apertures are different
            print(f"Unique apertures in this match: {apertures}")



In [21]:
DIR = "./contests/2/"

metadata_csv = pd.read_csv(DIR+"metadata.csv")
# sort by CreateDate
metadata_csv = metadata_csv.sort_values("CreateDate")

metadata_csv

,FileName,Model,CreateDate,Match,Orientation,tags,Composite:AvgBitrate,Composite:GPSAltitude,Composite:GPSAltitudeRef,Composite:GPSLatitude,...,QuickTime:TrackLayer,QuickTime:TrackModifyDate,QuickTime:TrackVolume,QuickTime:TransferCharacteristics,QuickTime:VideoFrameRate,QuickTime:VideoFullRangeFlag,QuickTime:XResolution,QuickTime:YResolution,SourceFile,Tags2
0,IMG_1889.MOV,iPhone 16 Pro Max,2025:02:03 05:46:41,4,90,"['광각','자연광','인물']",8555811,35.995,0.0,37.4199,...,0,2025:02:03 05:46:49,0,NaN,30.000000,NaN,72,72,./data/contests/2/IMG_1889.MOV,[]
1,20250203_144640.mp4,Galaxy S25 Ultra,2025:02:03 05:46:50,4,90,"['광각','자연광','인물']",10204209,NaN,NaN,37.4199,...,0,2025:02:03 05:46:50,0,1.0,30.046088,0.0,72,72,./data/contests/2/20250203_144640.mp4,[]
2,20250203_150202.mp4,Galaxy S25 Ultra,2025:02:03 06:02:08,12,90,"['망원','자연광','풍경']",9883644,NaN,NaN,37.4198,...,0,2025:02:03 06:02:08,0,1.0,30.001587,0.0,72,72,./data/contests/2/20250203_150202.mp4,[]
3,IMG_1892.MOV,iPhone 16 Pro Max,2025:02:03 06:02:14,12,90,"['망원','자연광','풍경']",8381324,52.251,0.0,37.4197,...,0,2025:02:03 06:02:19,0,NaN,29.978888,NaN,72,72,./data/contests/2/IMG_1892.MOV,[]
4,IMG_1893.MOV,iPhone 16 Pro Max,2025:02:03 06:03:46,5,90,"['광각','자연광','손떨방']",8558210,52.128,0.0,37.4197,...,0,2025:02:03 06:03:54,0,NaN,30.000000,NaN,72,72,./data/contests/2/IMG_1893.MOV,[]
5,20250203_150345.mp4,Galaxy S25 Ultra,2025:02:03 06:03:53,5,90,"['광각','자연광','손떨방']",9927771,NaN,NaN,37.4197,...,0,2025:02:03 06:03:53,0,1.0,30.014315,0.0,72,72,./data/contests/2/20250203_150345.mp4,[]
6,IMG_1907.MOV,iPhone 16 Pro Max,2025:02:03 08:27:36,6,90,"['광각','실내','인물']",8284364,107.366,0.0,37.4129,...,0,2025:02:03 08:27:45,0,NaN,30.000000,NaN,72,72,./data/contests/2/IMG_1907.MOV,[]
7,20250203_172735.mp4,Galaxy S25 Ultra,2025:02:03 08:27:46,6,90,"['광각','실내','인물']",9992965,NaN,NaN,37.4133,...,0,2025:02:03 08:27:46,0,1.0,30.018313,0.0,72,72,./data/contests/2/20250203_172735.mp4,[]
8,IMG_1916.MOV,iPhone 16 Pro Max,2025:02:03 08:44:50,11,90,"['줌','실내']",8264940,106.664,0.0,37.4131,...,0,2025:02:03 08:45:02,0,NaN,29.987112,NaN,72,72,./data/contests/2/IMG_1916.MOV,[]
9,20250203_174518.mp4,Galaxy S25 Ultra,2025:02:03 08:45:32,11,90,"['줌','실내']",9915120,NaN,NaN,37.4134,...,0,2025:02:03 08:45:32,0,1.0,29.907088,0.0,72,72,./data/contests/2/20250203_174518.mp4,[]


In [ ]:
metadata_csv["Model"] = metadata_csv["Model"].replace("SM-S938N", "Galaxy S25 Ultra")
metadata_csv.to_csv(DIR+"metadata.csv", index=False)